# Finite volume solver for the shallow water equations

We consider the shallow water equations as an example of a nonlinear conservation law, i.e. we consider

$$
  \partial_t \mathbf{U} + \operatorname{div}(\mathbf{F} (\mathbf{U} )) = 0 \qquad in \qquad \Omega \times[0,T],
$$
with 
$$
\mathbf{U} = (h, hu) = (\mathbf{u}_1, \mathbf{u}_2)
$$
and 
$$
  \mathbf{F}(\mathbf{U})) 
  = \left( \begin{array}{c} h u\\ h u^2 + \frac12 g h^2 \end{array} \right) 
  = \left( \begin{array}{c} \mathbf{u}_2\\ \frac{\mathbf{u}_2^2}{\mathbf{u}_1} + \frac12 g \mathbf{u}_1^2 \end{array} \right)
$$

## Jacobian of the flux for shallow water:
$$
\mathbf{A}(\mathbf{u}) =
\left(
      \begin{array}{cc}
      0 & 1 \\
      - \frac{\mathbf{u}_2^2}{\mathbf{u}_1^2} + g \mathbf{u}_1 & 2 \frac{\mathbf{u}_2}{\mathbf{u}_1} 
      \end{array}
\right)
=
\left(
      \begin{array}{cc}
      0 & 1 \\
      - u^2 + g h & 2 u
      \end{array}
\right)
\quad \rho(\mathbf{A}(\mathbf{u})) = \{ u - \sqrt{gh}, u + \sqrt{gh} \}
$$

## Prerequisites (libraries and helper functions)
We start loading several libraries:

In [ ]:
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
ngsglobals.msg_level = 1
from ngsolve.meshes import *
from draw import *

## The mesh, parameters and boundary conditions.

In [ ]:
N=32
mesh = Make1DMesh(n=N,periodic=True)
g = 9.81
k=0
CFL=1
dt=1.0/6*CFL/N #(6 is an educated guess on the maximum characteristic speed)
lambd=1

In [ ]:
dim = mesh.dim+1

## A simple finite volume solver

We can now define a very simple finite volume solver based on:
 * a flux function $F$,
 * a numerical flux function $\hat{f}_n$,
 * initial values $u_0$,
 * a mesh and
 * a time step for an explicit Euler discretization.
 
Let $u_T$ be the constant corresponding to one element. Then the scheme reads as

$$
u_T^{n+1} = u_{T}^n - \frac{\Delta t}{|T|} \sum_{K \in \partial T} 
\hat{f}_{K,n}(u_T,u_{T'},n_K) |K|
$$
    

This, together with some plotting at $t = 0.2,..,1$ is done in the following function:

In [ ]:
W = L2(mesh,order=k,dim=dim)
def Solve(F, fhatn, u0, mesh, dt):
    gfu=GridFunction(W)
    U,V = W.TnT()
    a = BilinearForm (W, nonassemble=True)
    a += InnerProduct(fhatn(F,U,U.Other(),specialcf.normal(mesh.dim)),V) * dx(element_boundary=True)
    t=0

    gfu.Set(u0)
    Ts = [i*0.005 for i in range(1,5)] + [i*0.025 for i in range(1,21)]
    i = 0
    for T in Ts:
        while t < T-dt/2:
            gfu.vec.data -= dt * W.InvM() @ a.mat * gfu.vec
            t += dt
            i += 1
            Redraw()
        Draw1D(mesh,[(gfu[0],"h")],n_p=k+1)
        print("t =",t," total mass:",Integrate(gfu[0],mesh))
    print(i,"steps")
    return gfu

## The flux function, initial values and boundary conditions

In [ ]:
def F(U):
    h, hv = U   # unpack the state vector
    return None # TODO

As initial values we prescribe $u_0(x)$:

In [ ]:
def tanh(x):
    return (exp(x)-exp(-x)) / (exp(x)+exp(-x))
def smoothed_jump(leftval, rightval, jumpposition, layerwidth):
    return leftval + (0.5*tanh(4/layerwidth*(x-jumpposition))+0.5)*(rightval-leftval)
h0 = smoothed_jump(0,1,0.4,0.1)+smoothed_jump(1,0,0.6,0.1)
Draw1D(mesh,[(h0,"$h_0$")])
U0 = CoefficientFunction((h0,0))

## numerical examples

In [ ]:
def Abs(u):
    return IfPos(u,u,-u)
def Max(u,v):
    return IfPos(u-v,u,v)

### discretizations with a Lax-Friedrichs and Roe flux

In [ ]:
def fhatn_central(F,u1,u2,n):
    return 0.5*F(u1)*n+0.5*F(u2)*n

def fhatn(F,u1,u2,n): # TODO
    return None
gfu = Solve(F,fhatn, U0, mesh, dt)